# Breast cancer neural network, porém com kfold

In [1]:
import pandas as pd

previsores_df = pd.read_csv("entradas_breast.csv")
classes_df    = pd.read_csv("saidas_breast.csv")

# Primeiro passo

Vamos iniciar a rede neural, da mesma forma que fizemos no primeiro exemplo, porém estamos adicionando também um **Dropout**, uma layer com funcionalidade de "dropar" certos valores para 0 para reduzir overfitting no modelo

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam 

def create_neural_network():
    num_elementos = len(previsores_df.columns)
    num_neuronios = round((num_elementos + 1)/2)

    classificador = Sequential()

    camada_dos_atributos = Dense(units=num_neuronios,
                            activation="relu",
                            kernel_initializer="random_uniform",
                            input_dim=num_elementos)

    regulador_overfitt = Dropout(0.2)

    camada_dos_atributos2 = Dense(units=num_neuronios,
                            activation="sigmoid",
                            kernel_initializer="random_uniform",
                            input_dim=num_elementos)

    regulador_overfitt2 = Dropout(0.2)

    """
    Como a resposta é apenas se o câncer é benigno ou maligno, temos apenas
    um neurônio nessa camada. Além disso, como queremos uma de 0 ou 1, então
    utilizamos a função sigmoid, que irá gerar um número entre 0 e 1, logo
    nos dizendo a probabilidade do tipo de cancer
    """
    camada_de_respostas = Dense(units=1,
                                activation="sigmoid")

    classificador.add(camada_dos_atributos)
    classificador.add(regulador_overfitt)
    classificador.add(camada_dos_atributos2)
    classificador.add(regulador_overfitt2)
    classificador.add(camada_de_respostas)

    optimizer = Adam(learning_rate=0.001, beta_1=0.0001, beta_2=0.5)

    classificador.compile(optimizer=optimizer,
                        loss="binary_crossentropy",
                        metrics=["binary_accuracy"]
                        )

    return classificador

# Segundo passo

Vamos introduzir a validação cruzada

In [8]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

classificador = KerasClassifier(build_fn=create_neural_network,
                                epochs=500,
                                batch_size=10)

resultados = cross_val_score(estimator=classificador,
                            X=previsores_df,
                            y = classes_df, 
                            cv =10,
                            scoring="accuracy")

print(resultados)

Epoch 1/500


/tmp/ipykernel_1075948/1292833906.py:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classificador = KerasClassifier(build_fn=create_neural_network,


52/52 [==============================] - 0s 494us/step - loss: 0.6374 - binary_accuracy: 0.6836
Epoch 2/500
52/52 [==============================] - 0s 503us/step - loss: 0.5667 - binary_accuracy: 0.6934
Epoch 3/500
52/52 [==============================] - 0s 493us/step - loss: 0.5340 - binary_accuracy: 0.7441
Epoch 4/500
52/52 [==============================] - 0s 518us/step - loss: 0.4972 - binary_accuracy: 0.7637
Epoch 5/500
52/52 [==============================] - 0s 527us/step - loss: 0.5002 - binary_accuracy: 0.7734
Epoch 6/500
52/52 [==============================] - 0s 495us/step - loss: 0.4646 - binary_accuracy: 0.7891
Epoch 7/500
52/52 [==============================] - 0s 500us/step - loss: 0.5004 - binary_accuracy: 0.7773
Epoch 8/500
52/52 [==============================] - 0s 514us/step - loss: 0.4602 - binary_accuracy: 0.8066
Epoch 9/500
52/52 [==============================] - 0s 525us/step - loss: 0.4492 - binary_accuracy: 0.8105
Epoch 10/500
52/52 [====================

# Teceiro passo

Por fim, vamos imprimir o desvio e a média

In [9]:
print(resultados.mean())
print(resultados.std())

0.8700814536340852
0.07605443336191718
